### Урок 3. Классификация. Логистическая регрессия

In [1]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2,  1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [3]:
X

array([[1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00,
        1.00e+00, 1.00e+00, 1.00e+00, 1.00e+00],
       [1.00e+00, 1.00e+00, 2.00e+00, 5.00e+00, 3.00e+00, 0.00e+00,
        5.00e+00, 1.00e+01, 1.00e+00, 2.00e+00],
       [5.00e+02, 7.00e+02, 7.50e+02, 6.00e+02, 1.45e+03, 8.00e+02,
        1.50e+03, 2.00e+03, 4.50e+02, 1.00e+03],
       [1.00e+00, 1.00e+00, 2.00e+00, 1.00e+00, 2.00e+00, 1.00e+00,
        3.00e+00, 3.00e+00, 1.00e+00, 2.00e+00]])

In [4]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

In [5]:
def calc_std_fit(x):
    for i in range(x.shape[0]):
        if x[i].std() == 0:
            x[i]= np.zeros(x[i].shape)
        else:
            x[i] = (x[i] - x[i].mean()) / x[i].std()
    return x

In [6]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [7]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    err = 0
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
#        if i % (iterations / 10) == 0:
#            print(i, W, err)
    return W, calc_logloss(y, y_pred)

## Практическое задание

1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [8]:
def calc_logloss(y, y_pred):
    y_pred = y_pred.astype(np.float64)
    y_pred[y_pred == 0.0 ] = 0.00000001
    y_pred[y_pred == 1.0 ] = 0.99999999
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    err = np.sum(err)
    return err

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [9]:
np.random.seed(42)
alphas = np.array([(i + 1) for i in range(10)])
iters = np.array([(i + 1) * 1000 for i in range(10)])
best_error = np.inf
best_params = {}

for iter in iters:
    for alpha in alphas:
        err = eval_model(calc_std_fit(X), y, iterations=iter, alpha=alpha)[1]
        if err < best_error:
            best_error = err
            best_params = {'iter': iter, 
                           'alpha': alpha}

print('оптимальная ошибка ', best_error)
print('оптимальные параметры ', best_params)

оптимальная ошибка  0.07921746104141006
оптимальные параметры  {'iter': 10000, 'alpha': 10}


3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred_proba.

In [10]:
def calc_pred_proba(W, X):
    X_st = calc_std_fit(X)
    z = np.dot(W, X_st)
    y_pred_proba = sigmoid(z)
    return y_pred_proba

In [11]:
np.round(calc_pred_proba(W=eval_model(X, y, iterations=best_params['iter'], alpha=best_params['alpha'])[0], X=X), 2)

array([0.05, 0.  , 1.  , 0.  , 0.79, 0.16, 1.  , 0.  , 0.71, 1.  ])

4. Создайте функцию calc_pred, возвращающую предсказанный класс. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred.

In [12]:
def calc_pred (W, X):
    y_pred_proba = calc_pred_proba(W=W, X=X)
    y_pred = np.where(y_pred_proba <= 0.5, 0, 1)
    return y_pred

In [13]:
y_pred = calc_pred (W=eval_model(X, y, iterations=best_params['iter'], alpha=best_params['alpha'])[0], X=X)
y_pred

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [14]:
def accuracy(y, y_pred):
    accuracy = np.mean(y == y_pred)
    return accuracy

In [15]:
acc = accuracy(y, y_pred)
acc

1.0

In [16]:
def confusion_mx(y, y_pred):
    conf_mx = np.zeros((2, 2))
    for i in range(len(y)): 
        if y[i] == y_pred[i] == 1: # TP
            conf_mx[0][0] += 1
        elif y[i] == y_pred[i] == 0: # TN
            conf_mx[1][1] += 1
        elif y[i] != y_pred[i] and y[i] == 1: # FN
            conf_mx[1][0] += 1
        elif y[i] != y_pred[i] and y[i] == 0: # FP
            conf_mx[0][1] += 1
    return conf_mx

In [17]:
conf_mx = confusion_mx(y, y_pred)
conf_mx

array([[5., 0.],
       [0., 5.]])

In [18]:
def precision_recall(y, y_pred):
    res = {}
    conf_mx = confusion_mx(y, y_pred)
    TP = conf_mx[0][0]
    TN = conf_mx[1][1]
    FP = conf_mx[0][1]
    FN = conf_mx[1][0]
    res['precision'] = TP / (TP + FP)
    res['recall'] = TP / (TP + FN)
    return res

In [19]:
precision_recall(y, y_pred)

{'precision': 1.0, 'recall': 1.0}

In [20]:
def f_one_score(y, y_pred):
    return 2 * precision_recall(y, y_pred)['precision'] * precision_recall(y, y_pred)['recall'] / (precision_recall(y, y_pred)['precision'] + precision_recall(y, y_pred)['recall'])

In [21]:
f_one_score(y, y_pred)

1.0

6. Могла ли модель переобучиться? Почему?

Наша модель не могла переобучиться, так как является простой: в ней к существующим признакам не добавлялись новые их перемноженные комбинации или возведенные в степень - т.е. границей, которая разделяет (классифицирует) объекты, является обычной прямой линией (без изгибов)